# Quantitative Value Trading Strategy
#### This program selects the 50 stocks with the best intrinsic value metrics and calculates the recommended trades for an equal-weighted portfolio of these stocks.

In [90]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy.stats import percentileofscore as score
from statistics import mean
import math

In [7]:
stocks = pd.read_csv('CSV_files/sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [6]:
IEX_CLOUD_API_TOKEN = 'pk_682c76a7979e43349b214ad64d6d41b7'

In [13]:
def chunks(lst, n):
    """
    Yield successive n-sized chunks from lst.
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

## First Value Strategy
#### Low intrinsic value stocks are filtered out solely by Price-to-Earnings ratio.

In [30]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,117.50,25.88,N/A
1,AAL,14.86,3.46,N/A
2,AAP,67.58,10.04,N/A
3,AAPL,177.07,29.76,N/A
4,ABBV,149.24,30.64,N/A
...,...,...,...,...
496,YUM,129.10,26.08,N/A
497,ZBH,115.17,47.99,N/A
498,ZBRA,269.25,21.56,N/A
499,ZION,33.03,3.43,N/A


In [31]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)

In [32]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WYNN,94.240,1510.26,N/A
1,LVS,52.960,795.2,N/A
2,CRM,206.200,540.64,N/A
3,TAP,62.415,331.64,N/A
4,WELL,79.945,329.67,N/A
5,KSU,293.590,279.61,N/A
6,NVDA,456.335,237.67,N/A
7,VTR,41.770,218.46,N/A
8,STE,222.405,186.89,N/A
9,DXCM,105.620,126.28,N/A


In [33]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Please enter an integer.')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()

Enter the size of your portfolio: 1000000


In [36]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,WYNN,94.240,1510.26,212
1,LVS,52.960,795.2,377
2,CRM,206.200,540.64,96
3,TAP,62.415,331.64,320
4,WELL,79.945,329.67,250
5,KSU,293.590,279.61,68
6,NVDA,456.335,237.67,43
7,VTR,41.770,218.46,478
8,STE,222.405,186.89,89
9,DXCM,105.620,126.28,189


## Second Value Strategy
#### This strategy filters out "glamour" stocks based on percentiles of Price-to-Earnings ratio, Price-to-Book ratio, Price-to-Sales ratio, EV/EBITDA, and EV/GP.

In [118]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = my_columns)

In [119]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()   
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
            
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                '0',
                data[symbol]['quote']['peRatio'],
                '0',
                data[symbol]['advanced-stats']['priceToBook'],
                '0',
                data[symbol]['advanced-stats']['priceToSales'],
                '0',
                ev_to_ebitda,
                '0',
                ev_to_gross_profit,
                '0',
                '0'
            ],
            index = rv_columns),  
            ignore_index = True)
    

In [120]:
rv_dataframe = rv_dataframe.drop('Price-to-Earnings Ratio', axis = 1)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,117.68,0,25.92,0,6.07,0,4.9900,0,18.499682,0,9.541080,0,0
1,AAL,14.85,0,3.46,0,-2.26,0,0.1873,0,4.696440,0,0.614247,0,0
2,AAP,67.44,0,10.02,0,1.54,0,0.3620,0,6.424212,0,1.162786,0,0
3,AAPL,177.21,0,29.78,0,45.61,0,7.1600,0,22.555575,0,16.760511,0,0
4,ABBV,148.78,0,30.55,0,20.60,0,4.7300,0,11.491793,0,8.218842,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.08,0,26.08,0,-4.28,0,5.1600,0,20.026384,0,9.003282,0,0
497,ZBH,115.53,0,48.14,0,2.03,0,3.5000,0,16.083017,0,5.917498,0,0
498,ZBRA,269.60,0,21.59,0,4.59,0,2.5000,0,15.234695,0,6.198576,0,0
499,ZION,32.94,0,3.42,0,1.06,0,1.0900,0,2.744341,0,1.142607,0,0


In [121]:
rv_dataframe[rv_dataframe.isnull().any(axis =1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.020,0,65.69,0,NaN,0,NaN,0,NaN,0,NaN,0,0
28,ALXN,182.500,0,59.45,0,NaN,0,NaN,0,NaN,0,NaN,0,0
40,AON,323.970,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0
88,CERN,94.920,0,47.94,0,NaN,0,NaN,0,NaN,0,NaN,0,0
118,CTL,11.000,0,9.73,0,NaN,0,NaN,0,NaN,0,NaN,0,0
121,CTXS,103.900,0,39.96,0,NaN,0,NaN,0,NaN,0,NaN,0,0
135,DISCK,24.420,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0
164,ETFC,49.260,0,14.27,0,NaN,0,NaN,0,NaN,0,NaN,0,0
183,FISV,114.230,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0
185,FLIR,57.340,0,32.03,0,NaN,0,NaN,0,NaN,0,NaN,0,0


In [122]:
for column in [ 'Price-to-Earnings ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [123]:
metrics = {
    'Price-to-Earnings ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',  
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,117.68,0,25.92,0.630739,6.07,0.776447,4.9900,0.782435,18.499682,0.744511,9.541080,0.734531,0
1,AAL,14.85,0,3.46,0.0998,-2.26,0.061876,0.1873,0.013972,4.696440,0.06986,0.614247,0.017964,0
2,AAP,67.44,0,10.02,0.203593,1.54,0.241517,0.3620,0.047904,6.424212,0.131737,1.162786,0.073852,0
3,AAPL,177.21,0,29.78,0.718563,45.61,0.972056,7.1600,0.882236,22.555575,0.848303,16.760511,0.946108,0
4,ABBV,148.78,0,30.55,0.740519,20.60,0.954092,4.7300,0.774451,11.491793,0.365269,8.218842,0.648703,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.08,0,26.08,0.634731,-4.28,0.057884,5.1600,0.794411,20.026384,0.800399,9.003282,0.704591,0
497,ZBH,115.53,0,48.14,0.892216,2.03,0.341317,3.5000,0.632735,16.083017,0.610778,5.917498,0.429142,0
498,ZBRA,269.60,0,21.59,0.54491,4.59,0.641717,2.5000,0.51497,15.234695,0.566866,6.198576,0.45509,0
499,ZION,32.94,0,3.42,0.097804,1.06,0.136727,1.0900,0.202595,2.744341,0.01996,1.142607,0.06986,0


In [124]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,117.68,0,25.92,0.630739,6.07,0.776447,4.9900,0.782435,18.499682,0.744511,9.541080,0.734531,0.733733
1,AAL,14.85,0,3.46,0.0998,-2.26,0.061876,0.1873,0.013972,4.696440,0.06986,0.614247,0.017964,0.052695
2,AAP,67.44,0,10.02,0.203593,1.54,0.241517,0.3620,0.047904,6.424212,0.131737,1.162786,0.073852,0.139721
3,AAPL,177.21,0,29.78,0.718563,45.61,0.972056,7.1600,0.882236,22.555575,0.848303,16.760511,0.946108,0.873453
4,ABBV,148.78,0,30.55,0.740519,20.60,0.954092,4.7300,0.774451,11.491793,0.365269,8.218842,0.648703,0.696607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.08,0,26.08,0.634731,-4.28,0.057884,5.1600,0.794411,20.026384,0.800399,9.003282,0.704591,0.598403
497,ZBH,115.53,0,48.14,0.892216,2.03,0.341317,3.5000,0.632735,16.083017,0.610778,5.917498,0.429142,0.581238
498,ZBRA,269.60,0,21.59,0.54491,4.59,0.641717,2.5000,0.51497,15.234695,0.566866,6.198576,0.45509,0.544711
499,ZION,32.94,0,3.42,0.097804,1.06,0.136727,1.0900,0.202595,2.744341,0.01996,1.142607,0.06986,0.105389


In [125]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.510,0,0.43,0.093812,0.0456,0.063872,0.0934,0.001996,0.270279,0.015968,0.094878,0.001996,0.035529
1,LNC,25.325,0,-1.43,0.083832,0.9091,0.103792,0.2858,0.031936,-1.499020,0.013972,0.285802,0.003992,0.047505
2,AAL,14.850,0,3.46,0.0998,-2.2600,0.061876,0.1873,0.013972,4.696440,0.06986,0.614247,0.017964,0.052695
3,GM,32.930,0,4.59,0.107784,0.6388,0.077844,0.2697,0.025948,1.264977,0.017964,0.907877,0.043912,0.054691
4,WBA,26.435,0,-6.88,0.065868,1.0800,0.142715,0.1677,0.00998,-6.650243,0.011976,1.249922,0.083832,0.062874
5,XRX,15.530,0,-9.14,0.053892,0.7160,0.081836,0.3383,0.045908,5.248491,0.087824,0.969269,0.053892,0.064671
6,C,41.070,0,6.54,0.133733,0.4345,0.067864,0.6182,0.097804,3.686691,0.037924,0.652261,0.01996,0.071457
7,KSS,25.915,0,-157.54,0.011976,0.8532,0.093812,0.1768,0.011976,7.754208,0.185629,1.244909,0.081836,0.077046
8,SYF,32.025,0,5.92,0.117764,1.0900,0.147705,0.6775,0.101796,3.796525,0.03992,0.887267,0.041916,0.08982
9,WFC,41.530,0,10.38,0.211577,0.4819,0.071856,0.7679,0.121756,3.095571,0.025948,0.805127,0.033932,0.093014


In [96]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [126]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,FRC,3.510,5698,0.43,0.093812,0.0456,0.063872,0.0934,0.001996,0.270279,0.015968,0.094878,0.001996,0.035529
1,LNC,25.325,789,-1.43,0.083832,0.9091,0.103792,0.2858,0.031936,-1.499020,0.013972,0.285802,0.003992,0.047505
2,AAL,14.850,1346,3.46,0.0998,-2.2600,0.061876,0.1873,0.013972,4.696440,0.06986,0.614247,0.017964,0.052695
3,GM,32.930,607,4.59,0.107784,0.6388,0.077844,0.2697,0.025948,1.264977,0.017964,0.907877,0.043912,0.054691
4,WBA,26.435,756,-6.88,0.065868,1.0800,0.142715,0.1677,0.00998,-6.650243,0.011976,1.249922,0.083832,0.062874
5,XRX,15.530,1287,-9.14,0.053892,0.7160,0.081836,0.3383,0.045908,5.248491,0.087824,0.969269,0.053892,0.064671
6,C,41.070,486,6.54,0.133733,0.4345,0.067864,0.6182,0.097804,3.686691,0.037924,0.652261,0.01996,0.071457
7,KSS,25.915,771,-157.54,0.011976,0.8532,0.093812,0.1768,0.011976,7.754208,0.185629,1.244909,0.081836,0.077046
8,SYF,32.025,624,5.92,0.117764,1.0900,0.147705,0.6775,0.101796,3.796525,0.03992,0.887267,0.041916,0.08982
9,WFC,41.530,481,10.38,0.211577,0.4819,0.071856,0.7679,0.121756,3.095571,0.025948,0.805127,0.033932,0.093014


In [127]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_format = writer.book.add_format(
    {
        'num_format': '0.0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [128]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Price', dollar_format],
    'C':['Number of Shares to Buy', integer_format],
    'D':['Price-to-Earnings ratio', float_format],
    'E':['PE Percentile', percent_format],
    'F':['Price-to-Book Ratio', float_format],
    'G':['PB Percentile', percent_format],
    'H':['Price-to-Sales Ratio', float_format],
    'I':['PS Percentile', percent_format],
    'J':['EV/EBITDA', float_format],
    'K':['EV/EBITDA Percentile',percent_format],
    'L':['EV/GP', float_format],
    'M':['EV/GP Percentile', percent_format],
    'N':['RV Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()